In [9]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e24/sample_submission.csv
/kaggle/input/playground-series-s3e24/train.csv
/kaggle/input/playground-series-s3e24/test.csv


In [10]:

test_data = pd.read_csv('/kaggle/input/playground-series-s3e24/test.csv')
#print (test_data)

train_data = pd.read_csv('/kaggle/input/playground-series-s3e24/train.csv')
#print (train_data)


import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)


# Fit the model on all features of the DataFrame
#iso_forest.fit(test_data)

# Predict if a data point is an outlier
#outliers = iso_forest.predict(test_data)

# Add a new column for outlier detection
#test_data['outlier'] = outliers

# Filter the DataFrame to remove outliers
#test_data_filtered = test_data[test_data['outlier'] == 1].drop('outlier', axis=1)

iso_forest.fit(train_data)

# Predict if a data point is an outlier
outliers = iso_forest.predict(train_data)

# Add a new column for outlier detection
train_data['outlier'] = outliers

# Filter the DataFrame to remove outliers
train_data_filtered = train_data[train_data['outlier'] == 1].drop('outlier', axis=1)


train_data['BMI'] = train_data['weight(kg)'] / (train_data['height(cm)']/100)**2
test_data['BMI'] = test_data['weight(kg)'] / (test_data['height(cm)']/100)**2


train_data['GH'] = train_data['Gtp'] * (train_data['hemoglobin']**2)
test_data['GH'] = test_data['Gtp'] * (test_data['hemoglobin']**2)

train_data['GHDL'] = train_data['Gtp'] * (train_data['HDL']**2)
test_data['GHDL'] = test_data['Gtp'] * (test_data['HDL']**2)

########Binning#######################

# Define the bins
bins = [0, 18.5, 24.99, 29.99, 34.99, 39.99, float('inf')]
labels = [-1.0, 0.0, 1, 2, 3, 4]


# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42).fit(train_data)
kmeansTest = KMeans(n_clusters=3, random_state=42).fit(test_data)

# Create a new feature for the cluster labels
train_data['cluster_label'] = kmeans.labels_
test_data['cluster_label'] = kmeansTest.labels_


print ('Cluster labels' )
print (train_data['cluster_label'])
# Bin the BMI column
train_data['BMI_category'] = pd.cut(train_data['BMI'], bins=bins, labels=labels, include_lowest=True)
test_data['BMI_category'] = pd.cut(test_data['BMI'], bins=bins, labels=labels, include_lowest=True)



# Define the bins
bins = [0, 18.5, 45 , float('inf')]
labels = [0, 1, 2]

train_data['Age_category'] = pd.cut(train_data['BMI'], bins=bins, labels=labels, include_lowest=True)
test_data['Age_category'] = pd.cut(test_data['BMI'], bins=bins, labels=labels, include_lowest=True)

#########Binning#################

train_data['Cholesterol Ratio'] = train_data['HDL'] / train_data['LDL']
test_data['Cholesterol Ratio'] = test_data['HDL'] / test_data['LDL']

train_data['CRA'] = train_data['HDL'] / (train_data['LDL']**2)
test_data['CRA'] = test_data['HDL'] / (test_data['LDL']**2)

train_data['BMIAP'] = train_data['BMI'] / (train_data['age'] * train_data['age'] )
test_data['BMIAP'] = test_data['BMI'] / (test_data['age'] * test_data['age'])

train_data['BMIAD'] = (train_data['BMI']**2) / train_data['age'] 
test_data['BMIAD'] = (test_data['BMI']**2) / test_data['age']

train_data['HH'] = train_data['hemoglobin'] * (train_data['height(cm)']**3) 
test_data['HH'] = test_data['hemoglobin'] * (test_data['height(cm)']**3)

train_data['GHH'] = train_data['Gtp'] * (train_data['hemoglobin']**2) 
test_data['GHH'] = test_data['Gtp'] * (test_data['hemoglobin']**2) 

train_data['ABR'] = (train_data['AST'] / train_data['BMI']**2 ) * train_data['age']
test_data['ABR'] = (test_data['AST'] / test_data['BMI']**2 ) * test_data['age']
    
train_data['HW_Ratio'] = train_data['height(cm)'] / train_data['waist(cm)']
test_data['HW_Ratio'] = test_data['height(cm)'] / test_data['waist(cm)']

train_data['HA_Ratio'] = train_data['height(cm)'] / train_data['age']
test_data['HA_Ratio'] = test_data['height(cm)'] / test_data['age']

train_data['SA'] = train_data['systolic'] / (train_data['age'] **2)
test_data['SA'] = test_data['systolic'] / (test_data['age'] **2)

#train_data['LAA'] = train_data['cluster_label'] / (train_data['LDL'] **2)
#test_data['LAA'] = test_data['cluster_label'] / (test_data['LDL'] **2)

train_data['HA'] = train_data['BMI'] / (train_data['height(cm)'] **2)
test_data['HA'] = test_data['BMI'] / (test_data['height(cm)'] **2)

train_data['CHA'] = train_data['hemoglobin'] / (train_data['systolic'] **2)
test_data['CHA'] = test_data['hemoglobin'] / (test_data['systolic'] **2)

train_data['LA'] = train_data['LDL'] / (train_data['Gtp'] **3)
test_data['LA'] = test_data['LDL'] / (test_data['Gtp'] **3)



train_data['Gtp'] = np.clip(train_data['Gtp'], 0, 300)
train_data['HDL'] = np.clip(train_data['HDL'], 0, 110)
train_data['LDL'] = np.clip(train_data['LDL'], 0, 200)
train_data['ALT'] = np.clip(train_data['ALT'], 0, 150)
train_data['AST'] = np.clip(train_data['AST'], 0, 100)
train_data['serum creatinine'] = np.clip(train_data['serum creatinine'], 0, 3) 


test_data['Gtp'] = np.clip(test_data['Gtp'], 0, 300)
test_data['HDL'] = np.clip(test_data['HDL'], 0, 110)
test_data['LDL'] = np.clip(test_data['LDL'], 0, 200)
test_data['ALT'] = np.clip(test_data['ALT'], 0, 150)
test_data['AST'] = np.clip(test_data['AST'], 0, 100)
test_data['serum creatinine'] = np.clip(test_data['serum creatinine'], 0, 3)


cols_to_normalize = ['Cholesterol Ratio','HA_Ratio','cluster_label','SA','HA','HH','BMI','LDL', 'Gtp','AST','ALT' ,'hemoglobin', 'systolic', 'HDL', 'serum creatinine']

            
scaler = StandardScaler()
train_data[cols_to_normalize] = scaler.fit_transform(train_data[cols_to_normalize])
test_data[cols_to_normalize] = scaler.transform(test_data[cols_to_normalize])

x_train, x_test, y_train, y_test = train_test_split(train_data[cols_to_normalize], train_data['smoking'], test_size=0.2, random_state=42)


# For each feature apply IQR method
for col in cols_to_normalize:
    Q1 = train_data[col].quantile(0.25)
    Q3 = train_data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Filter out the outliers from the training data
    train_data = train_data[(train_data[col] >= lower_bound) & (train_data[col] <= upper_bound)]

from sklearn.ensemble import VotingClassifier

# Create the sub-models
clf1 = KNeighborsClassifier(n_neighbors=45)
clf2 = xgb.XGBClassifier(n_estimators=1300, learning_rate=.027240097672443227, gamma=0, subsample=0.9772188294492674, colsample_bytree=0.204278500652, max_depth=11)
clf3 = RandomForestClassifier(n_estimators=1350, max_depth = 150,min_samples_leaf=15,random_state=42,n_jobs=-1, class_weight='balanced')


#clf3 = LogisticRegression()

# Create the voting ensemble model (soft voting)
voting_clf = VotingClassifier(estimators=[('knn', clf1), ('xgb', clf2), ('lr', clf3)], voting='soft')

# Fit the model on the training data
voting_clf.fit(x_train, y_train)

# Predict and evaluate on the test data
pred_probs = voting_clf.predict_proba(x_test)[:, 1]
auc_score = roc_auc_score(y_test, pred_probs)
print(f"ROC AUC Score Voting: {auc_score}")

# Get feature importances
#feature_importances = clf.feature_importances_

# Create a dataframe for visualization
#features = x_train.columns
#importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Sort the dataframe by importance
#importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualize using seaborn's barplot
#plt.figure(figsize=(12, 8))
#sns.barplot(x='Importance', y='Feature', data=importance_df)
#plt.title('Random Forest Feature Importances')
#plt.show()

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels
0         0
1         0
2         0
3         0
4         0
         ..
159251    0
159252    0
159253    0
159254    0
159255    0
Name: cluster_label, Length: 159256, dtype: int32
ROC AUC Score Voting: 0.8514095504276453


In [5]:

# Make predictions using the meta_model
test_data['prediction'] = voting_clf.predict_proba(test_data[cols_to_normalize])[:, 1]

subset_data = test_data[['id', 'prediction']]

subset_data.to_csv("/kaggle/working/submission.csv", index=False)


In [12]:
#import torch
#import torch.nn as nn
#import torch.optim as optim
#import torch.nn.functional as F

# 2. Prepare Data for PyTorch
#from torch.utils.data import DataLoader, TensorDataset

# Convert data into PyTorch tensors
#x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
#x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
#y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

#train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
#test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=32)

# 3. Define the Neural Network
#class NeuralNet(nn.Module):
#    def __init__(self, input_size, dropout_prob=0.5):
#        super(NeuralNet, self).__init__()
#        self.fc1 = nn.Linear(input_size, 32)
#        self.fc2 = nn.Linear(32, 64)
#        self.fc3 = nn.Linear(64, 128)
##        self.dropout = nn.Dropout(dropout_prob)
#        self.fc4 = nn.Linear(128, 2)  # No need for softmax here
#
#    def forward(self, x):
 #       x = F.leaky_relu(self.fc1(x))
#        x = F.leaky_relu(self.fc2(x))
#        x = F.leaky_relu(self.fc3(x))
#        x = self.fc4(x)  # Output raw scores
#        return x

#model = NeuralNet(x_train.shape[1])

# 4. Define Loss Function and Optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. Train the Neural Network
#num_epochs = 200
#counter = 0
#for epoch in range(num_epochs):
#    print ('Epoch' + str(counter))
#    counter = counter+ 1
 #   for inputs, labels in train_loader:
 #       outputs = model(inputs)
 #       loss = criterion(outputs, labels.long())
        
  #      optimizer.zero_grad()
  #      loss.backward()
 #      optimizer.step()

# 6. Evaluate the Neural Network
#model.eval()
#with torch.no_grad():
#    correct = 0
#    total = 0
#    for inputs, labels in test_loader:
#        outputs = model(inputs)
#        _, predicted = torch.max(outputs.data, 1)
#        total += labels.size(0)
#        correct += (predicted == labels.long()).sum().item()

#    print('Accuracy: {} %'.format(100 * correct / total))

Epoch0
Epoch1
Epoch2
Epoch3
Epoch4
Epoch5
Epoch6
Epoch7
Epoch8
Epoch9
Epoch10
Epoch11
Epoch12
Epoch13
Epoch14
Epoch15
Epoch16
Epoch17
Epoch18
Epoch19
Epoch20
Epoch21
Epoch22
Epoch23
Epoch24
Epoch25
Epoch26
Epoch27
Epoch28
Epoch29
Epoch30
Epoch31
Epoch32
Epoch33
Epoch34
Epoch35
Epoch36
Epoch37
Epoch38
Epoch39
Epoch40
Epoch41
Epoch42
Epoch43
Epoch44
Epoch45
Epoch46
Epoch47
Epoch48
Epoch49
Epoch50
Epoch51
Epoch52
Epoch53
Epoch54
Epoch55
Epoch56
Epoch57
Epoch58
Epoch59
Epoch60
Epoch61
Epoch62
Epoch63
Epoch64
Epoch65
Epoch66
Epoch67
Epoch68
Epoch69
Epoch70
Epoch71
Epoch72
Epoch73
Epoch74
Epoch75
Epoch76
Epoch77
Epoch78
Epoch79
Epoch80
Epoch81
Epoch82
Epoch83
Epoch84
Epoch85
Epoch86
Epoch87
Epoch88
Epoch89
Epoch90
Epoch91
Epoch92
Epoch93
Epoch94
Epoch95
Epoch96
Epoch97
Epoch98
Epoch99
Epoch100
Epoch101
Epoch102
Epoch103
Epoch104
Epoch105
Epoch106
Epoch107
Epoch108
Epoch109
Epoch110
Epoch111
Epoch112
Epoch113
Epoch114
Epoch115
Epoch116
Epoch117
Epoch118
Epoch119
Epoch120
Epoch121
Epoch122
Epo